# Network and relations making

In [1]:
import subprocess
import os
import time
from yml_create_functions import make_yml, make_network_agents_yml

In [2]:
name_simulation = 'simulation_tutorial'

## Network Agent Parameters

The schema can be changed in schema/schema.py. Here, you can change the parameters of the default schema

In [3]:
network_agents_parameters = {
    "default": {
        "id_message": "NaN",
        "has_tv": "false",
        "cause": -1,
        "method": "NaN",

        "type": "dumb",
        "response": "NaN",
        "stance": "agree",
        "repost": "NaN",
        "parent_id": "NaN"

    },

    "DumbViewer": [
        {"weight": 2, "type": "dumb"},
        {"weight": 2, "type": "dumb", "has_tv": "true"}
    ],
    "HerdViewer": [
        {"weight": 2, "type": "herd", "stance": "against"},
        {"weight": 2, "type": "herd", "has_tv": "true"}
    ],
    "WiseViewer": [
        {"weight": 1, "type": "wise", "stance": "against"},
        {"weight": 1, "type": "wise", "has_tv": "true", "stance": "against"}
    ],

}

## Other Parameters

In [4]:
parameters = {
    "default_state": "{}",
    "load_module": "schema", # path from yml to schema
    "environment_agents": "[]",
    "environment_class": "schema.NewsEnvironmentAgent", # path from yml to environment class
    "environment_params": {
        "prob_neighbor_spread": 0.02,  # 006
        "prob_tv_spread": 0.006,  # 001
        "prob_neighbor_cure": 0.006,  # 001
        "prob_backsliding": 0.01,  # 003
        "prob_dead": 0.006,  # 001,
        "prob_repost": 0.8,
        "mean_time_connection": 50,
        "var_time_connection": 30
    },
    "interval": 1,
    "max_time": 100,
    "name": name_simulation,
    "network_params": {
        "generator": "barabasi_albert_graph",
        "n": 100,
        "m": 5
    },
    "num_trials": 1
}

# Prob responses parameters

In [5]:
# Define your probabilities here
prob_response = {"dumb": {"support": 0.4, "deny": 0.3, "question": 0, "comment": 0.2},
                "herd": {"support": 0.25, "deny": 0.25, "question": 0.25, "comment": 0.25},
                "wise": {"support": 0.2, "deny": 0.2, "question": 0.3, "comment": 0.3}
                }
types = list(prob_response.keys())
responses = list( prob_response[types[0]].keys() )

for type_i in types:
    for response in responses:
        name_i = f"prob_{type_i}_{response}"
        prob_i = prob_response[type_i][response]
        # Here, we set each probability
        parameters["environment_params"][name_i] = prob_i

## Simulation execution

In [6]:
parameters_i = parameters.copy()
network_agents_parameters_i = network_agents_parameters.copy()
data = make_yml(parameters_i)
data += make_network_agents_yml(network_agents_parameters_i)

yml_path = os.path.join('schema', '{name_simulation}.yml') # YML path
with open(yml_path, 'w') as file:
    file.write(data)

command = "soil"
start = time.time()

output = subprocess.check_output([command, yml_path])
end = time.time()
seconds_simulation = str(end-start)
print("SIMULATION'S SECONDS:", seconds_simulation)

SIMULATION'S SECONDS: 8.23389196395874


# Conversation Making

In [7]:
from get_data.get_data import get_pivoted_data
from post.templates import POST_TEMPLATE, REPLY_TEMPLATE, INSTRUCTIONS_TEMPLATE
from anytree import Node, RenderTree, LevelOrderIter
from post.post import Post
from IA.ia import send_prompt

## Output sqlite path

In [10]:
analysis_path = os.path.join('soil_output', name_simulation)
sql_table_path = f'{name_simulation}_trial_0.sqlite'

## Get Data

In [11]:
attributes = ['cause', 'method', 'response', 'stance', 'repost']

data = get_pivoted_data(analysis_path, sql_table_path, attributes)

## Define News

In [12]:
NEWS = 'Chilean Government Announces Ambitious Plan for Sustainable Energy Transition'
SHOW_TREE = True

## Create the Network's Tree

In [13]:
root = Post(0, message=NEWS, step=0, post_template=POST_TEMPLATE,
             reply_template=REPLY_TEMPLATE,
             instructions_template=INSTRUCTIONS_TEMPLATE, news=NEWS)

In [19]:
list_nodes = [root]
for i in range(len(data)):
    aux = data[i]
    owner = aux[0]
    step = int(aux[1])
    id_message = aux[2]
    parent = list_nodes[int(aux[3])]
    state = aux[4] #unused
    attr_dict = {}
    for index, attr in enumerate(attributes):
        index +=5
        attr_dict[attr] = aux[index] 
    
    ### Specific for repost attribute
    if attr_dict["repost"] == 'true':
        attr_dict["repost"] = True
    elif attr_dict["repost"] == 'false':
        attr_dict["repost"] = False
    else:
        print(data[i])
        print(attr_dict["repost"])
        raise ValueError("Error repost format")
    ### 

    message = ''

    list_nodes.append(Post(name=id_message, parent=parent, owner=owner, step=step, message=message, 
                             post_template=POST_TEMPLATE, reply_template=REPLY_TEMPLATE,
                             instructions_template=INSTRUCTIONS_TEMPLATE, news=NEWS,
                             **attr_dict
                             )
    )

## Associate a Written Post to each Message

In [20]:
from parameters import API_KEY
import openai
openai.api_key = API_KEY

In [21]:
list_messages = []
for node_i in LevelOrderIter(root):
    if node_i == root:
        continue
    elif node_i.repost:
        continue

    instructions, prompt = node_i.get_prompt(language='english', 
                min_caract=130, max_caract=250,
                user_description='average toxic and angry social media user')
    answer = send_prompt(instructions, prompt, temp=1.5, max_tokens=1000)
    
    node_i.set_message(answer)
    list_messages.append((node_i, prompt))

## Plot the Network in a Tree

In [23]:
if SHOW_TREE:
    for pre, _, node in RenderTree(root):
        if node.name == 0:
            print(f"{pre}NEWS: {message}")
            continue

        if node.repost:
            message = "repost"
        else:
            message = "'"+node.message+"'"
        print(f"{pre}{node.owner} (t={node.step}) {message}")

NEWS: 
├── 43 (t=31) 'Finally, some good news from the Chilean Govt! 🎉 They've announced an ambitious plan to transition to sustainable energy 🌱 This means more renewable resources and fewer 𝗖𝗢𝟮 𝗲𝗺𝗶𝘀𝘀𝗶𝗼𝗻𝘀! 💨🚫 Great for the planet 🌍 and for our wallets, as energy costs will decrease in the long run ⚡️ Keep it up, Chile! 🇨🇱 #SustainableEnergy4TheWin 👍🔋 #ByeByeFossilFuels 😎👋 #AboutTimeGovernmentGotItRight 🤦‍♂️👀'
│   ├── 50 (t=60) 'That's awesome news from the Chilean Govt! 🎉 Their ambitious plan to transition to sustainable energy means hello to more renewable resources and no more CO2 emissions! 💨🚫 It's great for the planet 🌍 and even better for our wallets, as energy costs will decrease in the long run. ⚡️ Way to go, Chile! 🇨🇱 #SustainableEnergy4TheWin 👍🔋 #ByeByeFossilFuels 😎👋 #AboutTimeGovernmentGotItRight 🤦‍♂️👀'
│   └── 24 (t=98) 'Woah, it's about darn time! Finally, some good news from the Chilean Government! 🎉 Their ambitious plan to transition to sustainable energy is lit! 🌱 No mor